In [ ]:
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c playground-series-s4e11

 88% 4.00M/4.55M [00:01<00:00, 4.51MB/s]
100% 4.55M/4.55M [00:01<00:00, 3.66MB/s]


In [ ]:
!unzip playground-series-s4e11.zip

Archive:  playground-series-s4e11.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
!kaggle datasets download sumansharmadataworld/depression-surveydataset-for-analysis

Dataset URL: https://www.kaggle.com/datasets/sumansharmadataworld/depression-surveydataset-for-analysis
License(s): CC0-1.0
  0% 0.00/42.6k [00:00<?, ?B/s]
100% 42.6k/42.6k [00:00<00:00, 1.43MB/s]


In [ ]:
!unzip depression-surveydataset-for-analysis.zip

Archive:  depression-surveydataset-for-analysis.zip
  inflating: final_depression_dataset_1.csv  


In [ ]:
!pip install optuna catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
original_data = pd.read_csv("final_depression_dataset_1.csv")

In [ ]:
original_data["Depression"] = original_data["Depression"].map({"Yes":1,"No":0})

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
import lightgbm as lgb
import optuna
from catboost import CatBoostClassifier,Pool
from sklearn.ensemble import RandomForestClassifier

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## **Process Data**

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
train = pd.concat([train,original_data])

In [ ]:
scale_pos_weight =  train["Depression"].value_counts().iloc[0] / train["Depression"].value_counts().iloc[1]

In [ ]:
scale_pos_weight

4.50518791791561

In [ ]:
train = train.sample(frac=1).reset_index(drop=True)

In [ ]:
train.drop("id",axis=1,inplace=True)

In [ ]:
obj_col = train.select_dtypes("object").columns.tolist()
for col in obj_col:
  train[col] = train[col].astype("category")

In [ ]:
train = train.sample(frac=1).reset_index(drop=True)

In [ ]:
# NEXT STEP ENSEMBEL MODEL

## **ENSEMBEl MODEL**

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class ProcessCatFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, cat_features):
        self.cat_features = cat_features

    def fit(self, X, y=None):
        # No fitting needed, just return self
        return self

    def transform(self, X):
        # Process the categorical features
        X_copy = X.copy()
        X_copy[self.cat_features] = X_copy[self.cat_features].astype("object").fillna("missing")
        X_copy[self.cat_features] = X_copy[self.cat_features].astype("category")
        return X_copy

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline,Pipeline


def ensembelModel(x_train,y_train,obj_col,params):
  catmodel = CatBoostClassifier(nan_mode="Min",cat_features=obj_col,eval_metric="Accuracy",loss_function='Logloss',random_state=42,scale_pos_weight=4.50518791791561,task_type="GPU",verbose=0,**params["catboost"])

  catpipeline = make_pipeline(ProcessCatFeatures(obj_col),catmodel)

  estimators = [
      ('lgbm', LGBMClassifier(random_state=42,scale_pos_weight=4.50518791791561,objective='binary',metric= 'binary_logloss',boosting_type='gbdt',**params["lgbm"],verbosity=-1)),
      ('catboost', catpipeline),
      ('xgboost', XGBClassifier(enable_categorical=True,random_state=42,tree_method='gpu_hist',predictor='gpu_predictor',verbosity=0,scale_pos_weight=4.50518791791561,**params["xgb"]))
  ]

  clf = StackingClassifier(
      estimators=estimators, final_estimator=LogisticRegression()
  )
  return clf

In [ ]:
X = train.drop("Depression",axis=1)
Y = train["Depression"]

In [ ]:
from optuna import trial
import numpy as np
from sklearn.model_selection import cross_val_score
def objective(trial):
  params = {
      "xgb":{
          'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
          'max_depth': trial.suggest_int('max_depth', 3, 10),
          'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
          'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
          'subsample': trial.suggest_float('subsample', 0.5, 1.0),
          'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
          'max_cat_to_onehot': trial.suggest_int('max_cat_to_onehot', 1, 10),
          'max_cat_threshold': trial.suggest_int('max_cat_threshold', 32, 128),
          'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10.0, log=True),
          'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 10.0, log=True),
      },
      "catboost":{
              'iterations': trial.suggest_int('iterations', 100, 1000),
              'depth': trial.suggest_int('depth', 3, 10),
              'learning_rate': trial.suggest_float('cat_learning_rate', 0.01, 0.3, log=True),
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 10),
              'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.1, 10.0, log=True),
              'border_count': trial.suggest_int('border_count', 32, 128),

      },
      "lgbm":{
              'n_estimators': trial.suggest_int('lgb_n_estimators', 100, 1000),
              'max_depth': trial.suggest_int('lgb_max_depth', 3, 10),
              'learning_rate': trial.suggest_float('lgb_learning_rate', 0.01, 0.3, log=True),
              'min_child_samples': trial.suggest_int('min_child_samples', 1, 10),
              'subsample': trial.suggest_float('lgb_subsample', 0.5, 1.0),
              'colsample_bytree': trial.suggest_float('lgb_colsample_bytree', 0.5, 1.0),
              'max_bin': trial.suggest_int('lgb_max_bin', 32, 128),
              'reg_alpha': trial.suggest_float('lgb_reg_alpha', 0.1, 10.0, log=True),
              'reg_lambda': trial.suggest_float('lgb_reg_lambda', 0.1, 10.0, log=True),
              'cat_smooth': trial.suggest_float('cat_smooth', 1.0, 100.0),
      }
  }

  clf = ensembelModel(X,Y,obj_col,params)
  scores = cross_val_score(clf, X, Y, cv=5)
  return np.mean(scores)

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

[I 2024-11-20 09:39:58,019] A new study created in memory with name: no-name-ab3690f7-794f-485d-b385-b57b484d754e
[I 2024-11-20 10:02:54,337] Trial 0 finished with value: 0.9393603022618165 and parameters: {'n_estimators': 934, 'max_depth': 7, 'learning_rate': 0.02531449662871177, 'min_child_weight': 2, 'subsample': 0.8619353404698312, 'colsample_bytree': 0.6052963290916453, 'max_cat_to_onehot': 1, 'max_cat_threshold': 48, 'reg_alpha': 3.6365383024003837, 'reg_lambda': 4.623820904121375, 'iterations': 237, 'depth': 10, 'cat_learning_rate': 0.06667217182459025, 'min_data_in_leaf': 9, 'l2_leaf_reg': 5.041338475484808, 'border_count': 71, 'lgb_n_estimators': 767, 'lgb_max_depth': 9, 'lgb_learning_rate': 0.04150491577857082, 'min_child_samples': 3, 'lgb_subsample': 0.5746121056296347, 'lgb_colsample_bytree': 0.8238156201501892, 'lgb_max_bin': 106, 'lgb_reg_alpha': 0.41940497570987234, 'lgb_reg_lambda': 1.842350164792471, 'cat_smooth': 21.039936879313316}. Best is trial 0 with value: 0.9393

In [ ]:
ids = test["id"]
test.drop("id",axis=1,inplace=True)
obj_col = test.select_dtypes("object").columns.tolist()
for col in obj_col:
  test[col] = test[col].astype("category")


pred = clf.predict(test)
submmission = pd.DataFrame({"id":ids,"Depression":pred})
submmission.to_csv("submission.csv",index=False)

!kaggle competitions submit -c playground-series-s4e11 -f submission.csv -m "ensembel model"

100% 824k/824k [00:00<00:00, 1.41MB/s]
Successfully submitted to Exploring Mental Health Data